In [2]:
%load_ext autoreload
%autoreload 2

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
import sys
sys.path.append('../MOSS-RLHF')

import torch
from tqdm import tqdm
import importlib
from transformers import LlamaForCausalLM, LlamaTokenizer
from train_ppo import LlamaRewardModel, Llama
from ppo.ppo_datahelper import get_tokenizer


for i in range(torch.cuda.device_count()):
   print(torch.cuda.get_device_properties(i).name)
cuda = torch.device('cuda')
b1 = torch.tensor([1., 2.]).to(device=cuda)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
NVIDIA RTX A6000


In [1]:
# model_name_or_path1 = '/nas/ucb/henrypapadatos/MOSS-RLHF/models/moss-rlhf-policy-model-7B-en/recover'
model_name_or_path1 = '/nas/ucb/henrypapadatos/MOSS-RLHF/models/moss-rlhf-sft-model-7B-en/recover'
tokenizer_name_or_path = '/nas/ucb/henrypapadatos/MOSS-RLHF/models/moss-rlhf-policy-model-7B-en'

tokenizer = get_tokenizer(tokenizer_name_or_path)

model1 = Llama.from_pretrained(
    model_name_or_path1, 
    torch_dtype=torch.float32, 
    device_map="auto",
    opt=None,
    tokenizer=tokenizer)
# tokenizer = LlamaTokenizer.from_pretrained(tokenizer_name_or_path)

# model_name_or_path1 = 'baffo32/decapoda-research-llama-7B-hf'
# # model1 = LlamaForCausalLM.from_pretrained(model_name_or_path1, torch_dtype=torch.float16, device_map="auto")
# tokenizer = LlamaTokenizer.from_pretrained(model_name_or_path1)

print(model_name_or_path1)
print(model1.device)

NameError: name 'get_tokenizer' is not defined

In [2]:
# importlib.reload(train_ppo)
# importlib.reload(ppo.ppo_datahelper)
# import train_ppo
# import ppo.ppo_datahelper
from train_ppo import LlamaRewardModel, Llama
from ppo.ppo_datahelper import get_tokenizer

input_array = '''
Hey, my name is Alan and
'''
# input_array = ['''
# Hey, my name is Alan and
# ''',
# '''
# Hey, my name is Pete and
# '''
# ]

input_ids = tokenizer.encode(input_array, return_tensors='pt').to(device=cuda)
generated_text = model1.generate(input_ids, max_length=50, pad_token_id=tokenizer.eos_token_id, do_sample=True, temperature=0.01)
print(tokenizer.decode(generated_text[0], skip_special_tokens=True))
# print(input_ids)
# print(input_ids.shape)

# generated_text = model1.generate(input_ids, max_length=50, pad_token_id=tokenizer.eos_token_id, do_sample=True, temperature=1)
# token_sequences = generated_text[2][0]
# print(tokenizer.decode(token_sequences, skip_special_tokens=True))

ModuleNotFoundError: No module named 'train_ppo'

: 

In [40]:
# print(generated_text[2])

# token_sequences = generated_text[2][0]
# print(type(token_sequences))
# print(token_sequences.shape)
# print(token_sequences[:50])

print(tokenizer.decode(token_sequences, skip_special_tokens=True))

<class 'torch.Tensor'>
torch.Size([512])
tensor([29511, 22537, 20919, 13305, 24315,   303, 30067, 18251,  2653,  2328,
        30293, 31920, 22521, 22499, 15837, 16279, 13408, 10244,  8824, 19632,
        26174, 22019, 11563, 17183,  2512, 25248, 28189,   284,  6895,  2421,
        22779,   274,  1325, 20765, 29704, 12481,  6531, 25729, 14288,  6620,
        16442,  1244, 14310,  9511,  8617,   356, 13211,  3797, 17012, 16859],
       device='cuda:0')
utt categ aleinclude博Multi IIлькоode Посилання~~~~~~~~learnargumentserves nom SUланняGM入 Рес Cover Summermandeentialговéral loadingspiel Люслед vesünddomaindotnet modes Karl provincie Singap()));engl inlinefürEdLINiation пора Bekwrapper installay among leurs animation fuer datos répondtc Mens необtopcalendar appliesOccⁿ vuelhal quite crowd NAME react}{\leans ; disc urs bundle synchronLayoutanguageцен enabled Developház fooSet Can Agencyiaiauté mv Chine Limaonne Capitжуpectiontimestamp Brasile poz vitaabstracttranslate connexesrehhören res